In [34]:
import seaborn as sns
import pandas as pd

sube2023 = pd.read_csv("dataSets/dat-ab-usos-2023.csv")

print(sube2023.head())

  DIA_TRANSPORTE                                     NOMBRE_EMPRESA  \
0     2023-01-01  MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...   
1     2023-01-01  MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...   
2     2023-01-01                                 EMPRESA BATAN S.A.   
3     2023-01-01               COMPAÑIA  DE TRANSPORTE VECINAL S.A.   
4     2023-01-01           EMPRESA DE TRANSPORTE PERALTA RAMOS SACI   

              LINEA AMBA TIPO_TRANSPORTE JURISDICCION     PROVINCIA  \
0                 1   SI       COLECTIVO    MUNICIPAL  BUENOS AIRES   
1                2B   SI       COLECTIVO    MUNICIPAL  BUENOS AIRES   
2  BS_AS_LINEA 715M   NO       COLECTIVO    MUNICIPAL  BUENOS AIRES   
3   BS_AS_LINEA_326   SI       COLECTIVO   PROVINCIAL  BUENOS AIRES   
4   BS_AS_LINEA_512   NO       COLECTIVO    MUNICIPAL  BUENOS AIRES   

            MUNICIPIO  CANTIDAD DATO_PRELIMINAR  
0            MERCEDES        61              NO  
1            MERCEDES        11              N

Procesamiento de datos [2 pts.]
1. (a) Visualizar el tipo de datos de cada columna. Transformar la columna DIA_TRANSPORTE
para que sea reconocida como una fecha.
Sugerencia: investigar la función to_datetime de pandas. Para completar el argumento
format, revisar la documentación de datetime

In [35]:
sube2023["DIA_TRANSPORTE"] = pd.to_datetime(sube2023["DIA_TRANSPORTE"], format = '%Y-%m-%d')
#Verificamos que se convirtio en el tipo que queremos
type(sube2023["DIA_TRANSPORTE"][0])

pandas._libs.tslibs.timestamps.Timestamp

(b) Agregar tres columnas al DataFrame:

i. FECHA_DIA : debe indicar el nombre del día de la semana correspondiente a
DIA_TRANSPORTE

ii. FECHA_ORDINAL : debe indicar el ordinal correspondiente a DIA_TRANSPORTE (por
ejemplo, a 2023-01-01 le corresponde 1, a 2023-01-02 le corresponde 2 y así sucesiva-
mente). Debe ser un entero (int).

iii. FECHA_MES : debe indicar el mes correspondiente a DIA_TRANSPORTE
Sugerencia: investigar el método apply de DataFrame.

In [47]:
#Tomamos solo el prefijo del dia
sube2023["FECHA_DIA"] = [fecha.day_name()[0:3] for fecha in sube2023["DIA_TRANSPORTE"]]
print(sube2023.head())


  DIA_TRANSPORTE                                     NOMBRE_EMPRESA  \
0     2023-01-01  MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...   
1     2023-01-01  MUNICIPALIDAD DE MERCEDES PROVINCIA DE BUENOS ...   
2     2023-01-01                                 EMPRESA BATAN S.A.   
3     2023-01-01               COMPAÑIA  DE TRANSPORTE VECINAL S.A.   
4     2023-01-01           EMPRESA DE TRANSPORTE PERALTA RAMOS SACI   

              LINEA AMBA TIPO_TRANSPORTE JURISDICCION     PROVINCIA  \
0                 1   SI       COLECTIVO    MUNICIPAL  BUENOS AIRES   
1                2B   SI       COLECTIVO    MUNICIPAL  BUENOS AIRES   
2  BS_AS_LINEA 715M   NO       COLECTIVO    MUNICIPAL  BUENOS AIRES   
3   BS_AS_LINEA_326   SI       COLECTIVO   PROVINCIAL  BUENOS AIRES   
4   BS_AS_LINEA_512   NO       COLECTIVO    MUNICIPAL  BUENOS AIRES   

            MUNICIPIO  CANTIDAD DATO_PRELIMINAR FECHA_DIA  
0            MERCEDES        61              NO       Sun  
1            MERCEDES     

ii. FECHA_ORDINAL : debe indicar el ordinal correspondiente a DIA_TRANSPORTE (por
ejemplo, a 2023-01-01 le corresponde 1, a 2023-01-02 le corresponde 2 y así sucesiva-
mente). Debe ser un entero (int).

iii. FECHA_MES : debe indicar el mes correspondiente a DIA_TRANSPORTE
Sugerencia: investigar el método apply de DataFrame.